In [34]:
import requests
import json
import pandas as pd
from urllib.parse import quote

jj = lambda a, b: f"{a.rstrip('/')}/{b.lstrip('/')}"

def pp(arg):
    if isinstance(arg, str):
        arg = json.loads(arg)
    print(json.dumps(arg, indent=2, sort_keys=True))
    

API_URL = "http://localhost:8080/api/v1"
BASE_DIR = "/home/max/projects/autoware/src"

headers = {'Authorization': "Bearer TOKEN"}

In [42]:
def sym(path, symbol):
    r = requests.get(jj(API_URL, f"search?path={quote(path)}&symbol={quote(symbol)}"))
    r.raise_for_status()
    results = r.json()['results']
    records = []
    for path, matches in results.items():
        for match in matches:
            records.append((path, symbol, int(match['lineNumber']) - 1, match['line']))
    
    df = pd.DataFrame(records, columns=["path", "symbol", "line_num", "line"])
    return df


def list_subscriptions(path):
    #"results": {
    #"/universe/autoware.universe/planning/behavior_path_planner/include/behavior_path_planner/behavior_path_planner_node.hpp": [
    #  {
    #    "line": "  rclcpp::Subscription&lt;Odometry&gt;::SharedPtr <b>velocity_subscriber_</b>;",
    #    "lineNumber": "90"
    #  }
    #],
    syms_sub = sym(path, "create_subscription")
    for path, symb, line_num, line in syms_sub.itertuples(index=False, name=None):
        syms_bind = sym(path, "bind")
        print(f"{path:120s} {symb:25s} {line_num:4d} {line:120s}")
        closest_bind = syms_bind[syms_bind["line_num"]>=line_num][:1]
        if len(closest_bind) == 0:
            print(f"[WARN] Could not find matching bind statement for {path}#{symb}@{line_num}")
        for cpath, csymb, cline_num, cline in closest_bind.itertuples(index=False, name=None):
            print(f"{cpath:120s} {csymb:25s} {cline_num:4d} {cline:120s}")



list_subscriptions("")

/universe/autoware.universe/sensing/pointcloud_preprocessor/src/filter.cpp                                               create_subscription        139     sub_input_ = <b>create_subscription</b>&lt;PointCloud2&gt;(                                                         
[WARN] Could not find matching bind statement for /universe/autoware.universe/sensing/pointcloud_preprocessor/src/filter.cpp#create_subscription@139
/universe/autoware.universe/control/vehicle_cmd_gate/src/vehicle_cmd_gate.cpp                                            create_subscription         70   emergency_state_sub_ = this-&gt;<b>create_subscription</b>&lt;EmergencyState&gt;(                                     
/universe/autoware.universe/control/vehicle_cmd_gate/src/vehicle_cmd_gate.cpp                                            bind                        71     "input/emergency_state", 1, std::<b>bind</b>(&amp;VehicleCmdGate::onEmergencyState, this, _1));                     
/universe/autoware.universe/cont